In [1]:
import seaborn as sns
from numpy.linalg import solve
import math
import matplotlib.pyplot as plt
import numpy as np
import pylab
sns.set() 
import random

import scipy as stats
import scipy.special as special
from scipy.integrate import quad
import pandas as pd
import pprint
import csv
import scipy.special
import time


In [2]:
def Encode_Gn_code(u):  
    u=np.array(u)
    if (len(u)==1): 
        x=u;
    else:
        codd=(u[0:len(u)-1:2]+u[1:len(u):2])%2
        ceven=u[1:len(u):2]
        x=list(Encode_Gn_code(codd))+list(Encode_Gn_code(ceven)) 
    
    return x

In [3]:
def Do_Test(A):
    u=np.zeros(len(A))
    #print(A)
    for i in range(len(A)):
        if (A[i]==-1):
            if (random.random()<0.5):
                u[i]=1
        if (A[i]==0):
                u[i]=0
        if (A[i]==1):
                u[i]=1
                
    return u




def WBAWGN(y,sigma):
    return 2*y/2/sigma**2

def sigma(SNR): 
    sigma=math.sqrt(1/10**(SNR/10)/2)
    return sigma

def BAWGN_ch_for_2_users(u,v,sigma):
    u2=np.zeros(len(u))
    for i in range (len(u)):
        if (u[i]==0)and v[i]==0:
            u2[i]=2
        elif (u[i]==1)and v[i]==1:
            u2[i]=-2
        else:
            u2[i]=0
    f = np.random.normal(0, sigma, len(u)) 
    u2 = f + u2
    
    return u2

    
def L_count(y,sig):
    z=[]
    for i in range (len(y)):
        L_11 = -8*y[i]/(2*sig**2)
        L_01 = (-4*y[i]+4)/(2*sig**2) 
        L_10 = (-4*y[i]+4)/(2*sig**2)
        z.append(np.array([L_01, L_10, L_11]))
    return z
    

In [4]:
def decision(y):
    maximum = max(y)
    e=0
    for i in y:
        if i == maximum:
            return np.array([[int(e/2)], [e%2]])
        e+=1
        
def safe_exp(x):
    if (x>300):
        return np.exp(300)
    return np.exp(x)

def op_o(y1,y2):
    y = []
    
    for i in range(len(y1)):
        mass_ch = np.array([y1[i][0], y2[i][0], y1[i][2] + y2[i][1], y1[i][1] + y2[i][2]])
        maxs_ch = mass_ch.argmax()
        
        mass_znam = np.array([0, y1[i][0] + y2[i][0],y1[i][1] + y2[i][1],y1[i][2] + y2[i][2]])
        maxs_znam = mass_znam.argmax()
        znam = sum(np.exp(y1[i] + y2[i] - mass_znam[maxs_znam])) + 1/safe_exp(mass_znam[maxs_znam])
        
        L_01 = mass_ch[maxs_ch] - mass_znam[maxs_znam] + np.log((np.exp(y1[i][0] - mass_ch[maxs_ch]) + np.exp(y2[i][0] - mass_ch[maxs_ch]) + np.exp(y1[i][2] + y2[i][1]- mass_ch[maxs_ch]) + np.exp(y1[i][1] + y2[i][2]- mass_ch[maxs_ch])) / znam)
        
        mass_ch =  np.array([y1[i][1],y1[i][2] + y2[i][0],y2[i][1],y1[i][0] + y2[i][2]])
        maxs_ch = mass_ch.argmax()
        
        L_10 = mass_ch[maxs_ch] - mass_znam[maxs_znam] + np.log((np.exp(y1[i][1]- mass_ch[maxs_ch]) + np.exp(y1[i][2] + y2[i][0]- mass_ch[maxs_ch]) + np.exp(y2[i][1]- mass_ch[maxs_ch]) + np.exp(y1[i][0] + y2[i][2]- mass_ch[maxs_ch])) / znam)
        
        mass_ch = np.array([y1[i][2],y1[i][1] + y2[i][0],y1[i][0] + y2[i][1],y2[i][2]]) 
        maxs_ch = mass_ch.argmax()
        L_11 = mass_ch[maxs_ch] - mass_znam[maxs_znam] + np.log((np.exp(y1[i][2]- mass_ch[maxs_ch]) + np.exp(y1[i][1] + y2[i][0]- mass_ch[maxs_ch]) + np.exp(y1[i][0] + y2[i][1]- mass_ch[maxs_ch]) + np.exp(y2[i][2]- mass_ch[maxs_ch])) / znam)
        
        y.append(np.array([L_01,L_10,L_11] ))
    return y

def op_e(y1_n,y2,des):
    y1 = [np.array([1, y1_n[i][0] , y1_n[i][1], y1_n[i][2]]) for i in range(len(y1_n))]
    y = []
    #print(y2[0][0],y2[0][1],y2[0][2])
    #print(y1_n[0][0],y1_n[0][1],y1_n[0][2])
    
    for i in range(len(y1)):
        L_01 = y2[i][0] + y1[i][(int(des[0][i]) + 0) % 2 *2 + (int(des[1][i]) + 1) % 2 ] - y1[i][int(des[0][i]) % 2 *2 + int(des[1][i]) % 2 ]
        L_10 = y2[i][1] + y1[i][(int(des[0][i]) + 1) % 2 *2 + (int(des[1][i]) + 0) % 2 ] - y1[i][int(des[0][i]) % 2 *2 + int(des[1][i]) % 2 ]
        L_11 = y2[i][2] + y1[i][(int(des[0][i]) + 1) % 2 *2 + (int(des[1][i]) + 1) % 2 ] - y1[i][int(des[0][i]) % 2 *2 + int(des[1][i]) % 2 ]
        y.append(np.array([L_01,L_10,L_11] ))
    return y
        
        
def decoder_2_users_PC(y,A1,A2,sigma,u,v,n):   # на вход y_[np.array(L(01),L(10),L(11)] 
    
    
    if ( len(y)==1):
        if  A1 == -1  and  A2 == -1: 
            des = decision([0, y[0][0], y[0][1], y[0][2] ])
            u[n] = des[0]
            v[n] = des[1]
        if  A1 == -1 and A2 != -1:
            if A2 == 0 :
                des = decision( [0, y[0][1]])
            if A2 == 1 :
                des = decision( [ y[0][0] , y[0][2]])
            u[n] = des[0]
            v[n] = des[1]
        if  A1 != -1 and A2 == -1:
            if A2 == 0 :
                des = decision( [0, y[0][0] , ])
            if A2 == 1 :
                des = decision( [ y[0][1], y[0][2]])
            u[n] = des[0]
            v[n] = des[1]
        #print( [1, y[0][0], y[0][1], y[0][2] ])
        if A1 != -1 and A2 != -1:
            des = np.array([[A1[0]],[A2[0]]])
            u[n] = des[0]
            v[n] = des[1]
           
        return des
        
    L_O = op_o( y[0:len(y):2] , y[1:len(y):2] )
    L_O_des = decoder_2_users_PC(L_O, A1[0:int(len(A1)/2)], A2[0:int(len(A2)/2)] , sigma, u , v, 2*n)
    L_E = op_e(y[0:len(y):2] , y[1:len(y):2] , L_O_des) 
    L_E_des = decoder_2_users_PC(L_E, A1[int(len(A1)/2):len(A1)], A2[int(len(A2)/2):len(A2)] , sigma,u,v, 2*n+1)
    
    des=np.zeros((2,len(y)))
    for i in range(int (len(y)/2)):
        des[0][2*i] = (L_O_des[0][i] + L_E_des[0][i]) % 2
        des[0][2*i+1] = (L_E_des[0][i])
        des[1][2*i] = (L_E_des[1][i] + L_O_des[1][i]) % 2
        des[1][2*i+1] = (L_E_des[1][i])
    return des
        

In [8]:
def integrand1(u,m1,m2,s):
    return np.sqrt(1/(s**2*2*math.pi)*np.exp(-(u-m1)**2/(2*s**2))*np.exp(-(u-m2)**2/(2*s**2)))

def er_for_BAWGN(sigma):
    L=quad(integrand1 ,-np.inf , np.inf ,args=(1,-1,sigma))
    return L[0]


def Polar_Code_BAWGN_1_user(k , sigma, N):
    E = er_for_BAWGN(sigma)
    E1=np.array([1 - (1-E)**2, E**2])
    E=E1
    for i in range (1,N):
        E=E1
        E1=np.zeros(2*len(E))
        for j in range (len(E)):
            
            E1[2*j]=(1-(1-E[j])**2)
            E1[2*j+1]=E[j]**2
        #E = E.transpose()
        #Q = E.reshape(1, -1)
    E=E1
    #print(Q)
    #print(E)
    A_f_half = np.zeros(2**(N-1))
    #print(A_f_half)
    A_f_half[len(A_f_half)-1] = 1
    A_s_half = np.ones(2**(N-1))*(-1)
    errbound = np.sort(E[int(len(E)/2):len(E)])[k-1]
    for i in range(2**(N-1)):
        if (E[int(len(E)/2)+i]>errbound):
            A_s_half[i]=0
    return np.hstack((A_f_half,A_s_half))


def result(length, rate, snr, error):
    out = (str(length) + ";" + str(rate) + ";"  + str(snr) + ";" + str(error)+ ";" + "\n")
    with open("result_polar_{}_{}.txt".format(str(length),str(rate)), "a") as fout:
        fout.write(out)

In [11]:
def Test_2_users_high(n,R,SNR,bound):
    a = 0
    for lens in n:
        for rates in R:
            for SNRS in SNR:
                A1 = Polar_Code_BAWGN_1_user(int(rates*(2**lens)), sigma(SNRS),lens)
                A2 = Polar_Code_BAWGN_1_user(int(rates*(2**lens)), sigma(SNRS),lens)
                A2[int(len(A2)/2)-1] =  0
                print(a)
                a = a+1
                p=0
                itr = 1 
                while itr < bound and p < 50:
                    u = Do_Test(A1)
                    v = Do_Test(A2)
                    u1 = np.zeros(len(A1))
                    v1 = np.zeros(len(A2))
                    yer = BAWGN_ch_for_2_users(Encode_Gn_code(u),Encode_Gn_code(v),sigma(SNRS))
        
                    yer = L_count(yer,sigma(SNRS))
                    z = decoder_2_users_PC(yer,A1,A2,sigma(SNRS),u1,v1,0)
                    if list(u1)!=list(u) or list(v1)!=list(v) :
                        p += 1
                    itr += 1 
                result(2**lens,  rates, SNRS, p/itr)
    return p/itr

In [12]:
SNR = [-4.0,-3.5,-3,-2.5]
Rate = [0.2]
length = [6]

Test_2_users_high(length,Rate,SNR, 100000)




0
1
2
3


0.023934897079942556

In [15]:
data0 = pd.read_csv('result_polar_64_0.2.txt',sep = ";", names = ['Length', 'Rate','SNR', 'Error' , "Method"])

In [16]:
data0

,Length,Rate,SNR,Error,Method
0,64,0.2,-4.0,0.168919,BEC_1_user_with_one_in_the_middle
1,64,0.2,-3.5,0.109409,BEC_1_user_with_one_in_the_middle
2,64,0.2,-3.0,0.057803,BEC_1_user_with_one_in_the_middle
3,64,0.2,-2.5,0.023935,BEC_1_user_with_one_in_the_middle
